# Mining testimonial fragments of the Holocaust

**Experience domain:**

### Load the necessary libraries

In [1]:
import sys; sys.path.insert(0, '..')
import itertools

In [2]:
import get_topic_model_concordance as topic_concordancer
from utils import blacklab, db, text
mongo = db.get_db()

In [3]:
%config Completer.use_jedi = False
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
import random

### Helper functions

In [4]:
def create_contextual_query(lemmas,context_length=50):
    permutations = itertools.permutations(lemmas,len(lemmas))
    final_result = []
    for element in list(permutations):
        temp_result = []
        for el in element:
            temp_result.append('[lemma="'+el+'"]')
        temp_result = '('+('[]{0,'+str(context_length)+'}').join(temp_result)+')'
        final_result.append(temp_result)
    final_result = '|'.join(final_result)
    return final_result
        
        
            

In [5]:
from utils import blacklab, db, text
import requests
import json
def find_sentence_id(label):
    props = {'annotators': 'tokenize'}

    # set the encoding of the annotator
    requests.encoding = 'utf-8'
    # make a request
    r = requests.post('http://localhost:9000/', params={'properties':
                      json.dumps(props)},
                      data=label.encode('utf-8'))
    result = json.loads(r.text, encoding='utf-8')
    query = []
    for i, token in enumerate(result['tokens']):

        if ('...'in token['word'] and ((i == 0) or
           i == len(result['tokens']) - 1)):
            continue
        elif ('...'in token['word']):
            query.append('[]{0,50}')
        elif ('-'in token['word']):
            query.append('[]{0,3}')
        elif ("n't"in token['word']):
            query.append('[]')
        elif ("'re"in token['word']):
            query.append('[]')
        elif ("?"in token['word']):
            query.append('[]')
        elif ("."in token['word']):
            query.append('[]')
        elif ("'s"in token['word']):
            query.append('[]')
        elif (","in token['word']):
            query.append('[]')
        else:
            query.append('["' + token['word'] + '"]')

    query = ' '.join(query)
    try:
        sentence = blacklab.search_blacklab(query, window=0,
                                            lemma=False,
                                            include_match=True)
        token_end = sentence[0]['token_end']
        token_start = sentence[0]['token_start']
        print (sentence[0])
        mongo = db.get_db()
        results = mongo.tokens.find({'testimony_id':
                                    sentence[0]['testimony_id']},
                                    {'_id': 0})
        tokens = list(results)[0]['tokens']
        sentenceStart = tokens[token_start]['sentence_index']
        sentenceEnd = tokens[token_end]['sentence_index']
        originalsentence = sentence[0]['complete_match']
        return (sentenceStart,sentenceEnd,sentence[0]['testimony_id'])
    except:
        print("The following query returned a null result")
        print(query)
        
            


In [6]:
def create_parent_node(label):
    """Generate a root node for a tree structure."""
    testimony_id = random.randint(1, 20)
    node = {}
    node['label'] = label
    fragment = {'label': label,
                'essay_id': random.randint(1, 20),
                'tree': get_node(testimony_id, node, is_parent=True)}
    fragment['tree']['label'] = label

    return fragment

In [7]:
def get_node(testimony_id, node, is_parent=False):
    """Generate a parent or leaf node for a tree structure."""
    if is_parent:
        return {
            'label': node['label'],
            'testimony_id': random.randint(1, 20),
            'media_index': random.randint(1, 20),
            'media_offset': random.randint(1, 20),
            'start_sentence_index': random.randint(1, 20),
            'end_sentence_index': random.randint(1, 20),
            'children': [], }
    else:
        return {'label': node['label'],
                'testimony_id': node['testimony_id'],
                'media_index': float(node['media_index']),
                'media_offset': float(node['media_offset']),
                'start_sentence_index': float(node['start_sentence_index']),
                'end_sentence_index': float(node['end_sentence_index']),
                'children': [], }

In [8]:
def check_if_main_node_exist(node):
    results = mongo.fragments.find({'label':node},{'_id': 0})
    if len(results[0])==0:
        return False
    else:
        return True

In [9]:
def add_main_node(label):
    mongo.fragments.insert(create_parent_node(label))

In [10]:
def delete_main_node(label):
    mongo.fragments.delete_one({'label':label})

In [11]:
def add_testimonial_fragments(fragments):
    if check_if_main_node_exist(fragments['main_node']):
        results = mongo.fragments.find({'label':fragments['main_node']},{'_id':0})[0]
        mid_nodes = [element['label'] for element in results['tree']['children']]
        if fragments['mid_node'] in mid_nodes:
            print ("mid node exists cannot be added")
        else:
            
            mid_node = get_node('r',{'label':fragments['mid_node']},is_parent=True)
            for fragment in fragments['fragments']:
                leaf = get_node(fragment['testimony_id'],fragment)
                mid_node['children'].append(leaf)
            results['tree']['children'].append(mid_node)
            mongo.fragments.replace_one({'label':fragments['main_node']},results)

### Add the main node

In [12]:
main_node = "drive"
delete_main_node(main_node)
add_main_node(main_node)

/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.
  


### Set up the query

In [13]:
query = '[word="driven"]'

In [14]:
result = topic_concordancer.main(query,window=10,topicn=25)

Training lda model began
http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5Bword%3D%22driven%22%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=10
training of gensim corpus began
gensim corpus done


### Print the key topics

In [15]:
for i,element in enumerate(result['topic_documents']):
    print (i)
    topic_words =  element['topic_words'][1]
    print (topic_words)
    print ('\n')

0
0.500*"street" + 0.250*"people" + 0.250*"house" + 0.000*"way" + 0.000*"camp" + 0.000*"a_lot" + 0.000*"say" + 0.000*"work" + 0.000*"like" + 0.000*"know"


1
0.714*"german" + 0.143*"work" + 0.071*"seconds" + 0.071*"back" + 0.000*"way" + 0.000*"a_lot" + 0.000*"like" + 0.000*"drive_through" + 0.000*"know" + 0.000*"camp"


2
0.889*"train" + 0.111*"Jews" + 0.000*"work" + 0.000*"way" + 0.000*"camp" + 0.000*"a_lot" + 0.000*"house" + 0.000*"like" + 0.000*"know" + 0.000*"say"


3
0.609*"horse" + 0.348*"Germans" + 0.043*"people" + 0.000*"work" + 0.000*"camp" + 0.000*"a_lot" + 0.000*"say" + 0.000*"drive_through" + 0.000*"see" + 0.000*"train"


4
0.435*"school" + 0.304*"seconds" + 0.261*"work" + 0.000*"way" + 0.000*"camp" + 0.000*"a_lot" + 0.000*"see" + 0.000*"say" + 0.000*"like" + 0.000*"know"


5
0.684*"barracks" + 0.211*"ghetto" + 0.105*"time" + 0.000*"say" + 0.000*"camp" + 0.000*"a_lot" + 0.000*"drive_through" + 0.000*"work" + 0.000*"like" + 0.000*"know"


6
1.000*"horse" + 0.000*"say" + 0.00

### Analyze documents

In [16]:
i=0
for text in result['topic_documents'][i]['texts'][0:25]:
    print (text['matched_text_words'])
    print ('\n')

Put them everybody on the street , and they were driven to the ghetto . For example , into our house 


all -- we were taken onto the lorries and were driven out into the street . As I was passing , 


-- had to be dug by the people who were driven down the street , through -- to the -- to 


on the street we were reminded of how we were driven as cattle in the gutter under the Nazis . When 


by the German Nazis , was to guard the people driven out on the street so that they could not move 


women were taken to work or somewhere , we were driven in an unknown direction , but we knew what street 


And [ SIGHS ] ' A herd of humans are driven through deserted streets . A spark of hope in our 


were killed right in the street . And everybody was driven out . INT : And who was with you in 


the main street in the middle of the street , driven like cattle , with the valises , with the baggages 


center of the street where the cars and horses were driven , and they started to force us t

## Testimonial fragments

### 1.  

In [17]:
lemmas = ["town","drive"]

In [18]:
query = create_contextual_query(lemmas,context_length=10)
print (query)

([lemma="town"][]{0,10}[lemma="drive"])|([lemma="drive"][]{0,10}[lemma="town"])


In [19]:
domain_term = "town"

In [20]:
fragments = {}
fragments['main_node'] = main_node
fragments['mid_node'] = domain_term
fragments['fragments'] = []

In [21]:
fragment_1 = {}
fragment_1['original_sentence'] = "we were driven to this town with the Hungarian gendarme using two truncheons on old people who couldn't walk fast enough."
fragment_1['label']="(..) we were driven to this town with the Hungarian gendarme using two truncheons on old people who couldn't walk fast enough."
indices = find_sentence_id(fragment_1['original_sentence'])
fragment_1['start_sentence_index']=indices[0]
fragment_1['end_sentence_index']=indices[1]
fragment_1['media_offset'] = 0
fragment_1['media_index'] = 0
fragment_1['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_1)

http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22we%22%5D+%5B%22were%22%5D+%5B%22driven%22%5D+%5B%22to%22%5D+%5B%22this%22%5D+%5B%22town%22%5D+%5B%22with%22%5D+%5B%22the%22%5D+%5B%22Hungarian%22%5D+%5B%22gendarme%22%5D+%5B%22using%22%5D+%5B%22two%22%5D+%5B%22truncheons%22%5D+%5B%22on%22%5D+%5B%22old%22%5D+%5B%22people%22%5D+%5B%22who%22%5D+%5B%22could%22%5D+%5B%5D+%5B%22walk%22%5D+%5B%22fast%22%5D+%5B%22enough%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': "we were driven to this town with the Hungarian gendarme using two truncheons on old people who could n't walk fast enough . ", 'right': '', 'complete_match': "we were driven to this town with the Hungarian gendarme using two truncheons on old people who could n't walk fast enough . ", 'testimony_id': 'usc_shoah_4361', 'shelfmark': ['USC Shoah Foundation 4361'], 'token_start': 10702, 'token_end': 10725}


In [22]:
fragment_2 = {}
fragment_2['original_sentence'] = " the Jews were driven from all the surrounding little towns and villages"
fragment_2['label']="(..) the Jews were driven from all the surrounding little towns and villages (..)"
indices = find_sentence_id(fragment_2['original_sentence'])
fragment_2['start_sentence_index']=indices[0]
fragment_2['end_sentence_index']=indices[1]
fragment_2['media_offset'] = 0
fragment_2['media_index'] = 0
fragment_2['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_2)


http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22the%22%5D+%5B%22Jews%22%5D+%5B%22were%22%5D+%5B%22driven%22%5D+%5B%22from%22%5D+%5B%22all%22%5D+%5B%22the%22%5D+%5B%22surrounding%22%5D+%5B%22little%22%5D+%5B%22towns%22%5D+%5B%22and%22%5D+%5B%22villages%22%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': 'the Jews were driven from all the surrounding little towns and villages ', 'right': '', 'complete_match': 'the Jews were driven from all the surrounding little towns and villages ', 'testimony_id': 'irn508694', 'shelfmark': ['USHMM RG-50.462*0070'], 'token_start': 23436, 'token_end': 23448}


In [23]:
fragment_3 = {}
fragment_3['original_sentence'] = "first sense of brutality was when the Russian prisoners of war came through, were driven through the town and the treatment of them."
fragment_3['label']="(..)first sense of brutality was when the Russian prisoners of war came through, were driven through the town and the treatment of them."
indices = find_sentence_id(fragment_3['original_sentence'])
fragment_3['start_sentence_index']=indices[0]
fragment_3['end_sentence_index']=indices[1]
fragment_3['media_offset'] = 0
fragment_3['media_index'] = 0
fragment_3['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_3)

http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22first%22%5D+%5B%22sense%22%5D+%5B%22of%22%5D+%5B%22brutality%22%5D+%5B%22was%22%5D+%5B%22when%22%5D+%5B%22the%22%5D+%5B%22Russian%22%5D+%5B%22prisoners%22%5D+%5B%22of%22%5D+%5B%22war%22%5D+%5B%22came%22%5D+%5B%22through%22%5D+%5B%5D+%5B%22were%22%5D+%5B%22driven%22%5D+%5B%22through%22%5D+%5B%22the%22%5D+%5B%22town%22%5D+%5B%22and%22%5D+%5B%22the%22%5D+%5B%22treatment%22%5D+%5B%22of%22%5D+%5B%22them%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': 'first sense of brutality was when the Russian prisoners of war came through , were driven through the town and the treatment of them . ', 'right': '', 'complete_match': 'first sense of brutality was when the Russian prisoners of war came through , were driven through the town and the treatment of them . ', 'testimony_id': 'irn504577', 'shelfmark': ['USHMM RG-50.030*0082'], 'token_start': 9657, 'token_end': 9682}


In [24]:
fragment_4 = {}
fragment_4['original_sentence'] = "but it's enough if you are driven across one town."
fragment_4['label']= "(..) but it's enough if you are driven across one town."
indices = find_sentence_id(fragment_4['original_sentence'])
fragment_4['start_sentence_index']=indices[0]
fragment_4['end_sentence_index']=indices[1]
fragment_4['media_offset'] = 0
fragment_4['media_index'] = 0
fragment_4['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_4)


http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22but%22%5D+%5B%22it%22%5D+%5B%5D+%5B%22enough%22%5D+%5B%22if%22%5D+%5B%22you%22%5D+%5B%22are%22%5D+%5B%22driven%22%5D+%5B%22across%22%5D+%5B%22one%22%5D+%5B%22town%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': "but it 's enough if you are driven across one town . ", 'right': '', 'complete_match': "but it 's enough if you are driven across one town . ", 'testimony_id': 'usc_shoah_19550', 'shelfmark': ['USC Shoah Foundation 19550'], 'token_start': 27102, 'token_end': 27114}


In [25]:
add_testimonial_fragments(fragments)

### 2.  

In [26]:
lemmas = ["train","drive"]

In [27]:
query = create_contextual_query(lemmas,context_length=10)
print (query)

([lemma="train"][]{0,10}[lemma="drive"])|([lemma="drive"][]{0,10}[lemma="train"])


In [28]:
domain_term = "to the train"

In [29]:
fragments = {}
fragments['main_node'] = main_node
fragments['mid_node'] = domain_term
fragments['fragments'] = []

In [30]:
fragment_1 = {}
fragment_1['original_sentence'] = "And that morning, at 5 o'clock, the SS-- the Gestapo-- came, and they drove them all to the trains."
fragment_1['label']="And that morning, at 5 o'clock, the SS-- the Gestapo-- came, and they drove them all to the trains."
indices = find_sentence_id(fragment_1['original_sentence'])
fragment_1['start_sentence_index']=indices[0]
fragment_1['end_sentence_index']=indices[1]
fragment_1['media_offset'] = 0
fragment_1['media_index'] = 0
fragment_1['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_1)

http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22And%22%5D+%5B%22that%22%5D+%5B%22morning%22%5D+%5B%5D+%5B%22at%22%5D+%5B%225%22%5D+%5B%22o%27clock%22%5D+%5B%5D+%5B%22the%22%5D+%5B%22SS%22%5D+%5B%5D%7B0%2C3%7D+%5B%22the%22%5D+%5B%22Gestapo%22%5D+%5B%5D%7B0%2C3%7D+%5B%22came%22%5D+%5B%5D+%5B%22and%22%5D+%5B%22they%22%5D+%5B%22drove%22%5D+%5B%22them%22%5D+%5B%22all%22%5D+%5B%22to%22%5D+%5B%22the%22%5D+%5B%22trains%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': "And that morning , at 5 o'clock , the SS -- the Gestapo -- came , and they drove them all to the trains . ", 'right': '', 'complete_match': "And that morning , at 5 o'clock , the SS -- the Gestapo -- came , and they drove them all to the trains . ", 'testimony_id': 'usc_shoah_1641', 'shelfmark': ['USC Shoah Foundation 1641'], 'token_start': 2593, 'token_end': 2618}


In [31]:
fragment_2 = {}
fragment_2['original_sentence'] = "gathered the Jews and drove the Jews to the death camps and drove the trains "
fragment_2['label']="(..) gathered the Jews and drove the Jews to the death camps and drove the trains (..)"
indices = find_sentence_id(fragment_2['original_sentence'])
fragment_2['start_sentence_index']=indices[0]
fragment_2['end_sentence_index']=indices[1]
fragment_2['media_offset'] = 0
fragment_2['media_index'] = 0
fragment_2['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_2)


http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22gathered%22%5D+%5B%22the%22%5D+%5B%22Jews%22%5D+%5B%22and%22%5D+%5B%22drove%22%5D+%5B%22the%22%5D+%5B%22Jews%22%5D+%5B%22to%22%5D+%5B%22the%22%5D+%5B%22death%22%5D+%5B%22camps%22%5D+%5B%22and%22%5D+%5B%22drove%22%5D+%5B%22the%22%5D+%5B%22trains%22%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': 'gathered the Jews and drove the Jews to the death camps and drove the trains ', 'right': '', 'complete_match': 'gathered the Jews and drove the Jews to the death camps and drove the trains ', 'testimony_id': 'irn506747', 'shelfmark': ['USHMM RG-50.549.02*0020'], 'token_start': 3937, 'token_end': 3952}


In [32]:
fragment_3 = {}
fragment_3['original_sentence'] = "They told us, and they drove us to the station when the train was set."
fragment_3['label']="They told us, and they drove us to the station when the train was set."
indices = find_sentence_id(fragment_3['original_sentence'])
fragment_3['start_sentence_index']=indices[0]
fragment_3['end_sentence_index']=indices[1]
fragment_3['media_offset'] = 0
fragment_3['media_index'] = 0
fragment_3['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_3)

http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22They%22%5D+%5B%22told%22%5D+%5B%22us%22%5D+%5B%5D+%5B%22and%22%5D+%5B%22they%22%5D+%5B%22drove%22%5D+%5B%22us%22%5D+%5B%22to%22%5D+%5B%22the%22%5D+%5B%22station%22%5D+%5B%22when%22%5D+%5B%22the%22%5D+%5B%22train%22%5D+%5B%22was%22%5D+%5B%22set%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': 'They told us , and they drove us to the station when the train was set . ', 'right': '', 'complete_match': 'They told us , and they drove us to the station when the train was set . ', 'testimony_id': 'irn517852', 'shelfmark': ['USHMM RG-50.030*0500'], 'token_start': 44336, 'token_end': 44353}


In [33]:
fragment_4 = {}
fragment_4['original_sentence'] = " at least 50 Jewish people-- I knew quite a few of them-- who were driven to the railroad station from where they took the train"
fragment_4['label']= "(..) at least 50 Jewish people (..) who were driven to the railroad station from where they took the train (..)"
indices = find_sentence_id(fragment_4['original_sentence'])
fragment_4['start_sentence_index']=indices[0]
fragment_4['end_sentence_index']=indices[1]
fragment_4['media_offset'] = 0
fragment_4['media_index'] = 0
fragment_4['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_4)


http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22at%22%5D+%5B%22least%22%5D+%5B%2250%22%5D+%5B%22Jewish%22%5D+%5B%22people%22%5D+%5B%5D%7B0%2C3%7D+%5B%22I%22%5D+%5B%22knew%22%5D+%5B%22quite%22%5D+%5B%22a%22%5D+%5B%22few%22%5D+%5B%22of%22%5D+%5B%22them%22%5D+%5B%5D%7B0%2C3%7D+%5B%22who%22%5D+%5B%22were%22%5D+%5B%22driven%22%5D+%5B%22to%22%5D+%5B%22the%22%5D+%5B%22railroad%22%5D+%5B%22station%22%5D+%5B%22from%22%5D+%5B%22where%22%5D+%5B%22they%22%5D+%5B%22took%22%5D+%5B%22the%22%5D+%5B%22train%22%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': 'at least 50 Jewish people -- I knew quite a few of them -- who were driven to the railroad station from where they took the train ', 'right': '', 'complete_match': 'at least 50 Jewish people -- I knew quite a few of them -- who were driven to the railroad station from where they took the train ', 'testimony_id': 'usc_shoah_14168', 'shelfmark': ['USC Shoah Foundation 14168'], 'token_s

In [34]:
fragment_5 = {}
fragment_5['original_sentence'] = "people were driven into the trains, and the train left, "
fragment_5['label']= "(..) people were driven into the trains, and the train left (..)"
indices = find_sentence_id(fragment_5['original_sentence'])
fragment_5['start_sentence_index']=indices[0]
fragment_5['end_sentence_index']=indices[1]
fragment_5['media_offset'] = 0
fragment_5['media_index'] = 0
fragment_5['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_5)

http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22people%22%5D+%5B%22were%22%5D+%5B%22driven%22%5D+%5B%22into%22%5D+%5B%22the%22%5D+%5B%22trains%22%5D+%5B%5D+%5B%22and%22%5D+%5B%22the%22%5D+%5B%22train%22%5D+%5B%22left%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': 'people were driven into the trains , and the train left , ', 'right': '', 'complete_match': 'people were driven into the trains , and the train left , ', 'testimony_id': 'usc_shoah_27507', 'shelfmark': ['USC Shoah Foundation 27507'], 'token_start': 28375, 'token_end': 28387}


In [35]:
add_testimonial_fragments(fragments)

### 3.  

In [36]:
lemmas = ["street","drive"]

In [37]:
query = create_contextual_query(lemmas,context_length=10)
print (query)

([lemma="street"][]{0,10}[lemma="drive"])|([lemma="drive"][]{0,10}[lemma="street"])


In [38]:
domain_term = "through streets"

In [39]:
fragments = {}
fragments['main_node'] = main_node
fragments['mid_node'] = domain_term
fragments['fragments'] = []

In [40]:
fragment_1 = {}
fragment_1['original_sentence'] = "Jews were driven through the main street, the Praggerstrasser in Dresden, on their way to the station to be taken to the concentration camps"
fragment_1['label']="(..)Jews were driven through the main street, the Praggerstrasser in Dresden, on their way to the station to be taken to the concentration camps (..)"
indices = find_sentence_id(fragment_1['original_sentence'])
fragment_1['start_sentence_index']=indices[0]
fragment_1['end_sentence_index']=indices[1]
fragment_1['media_offset'] = 0
fragment_1['media_index'] = 0
fragment_1['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_1)

http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22Jews%22%5D+%5B%22were%22%5D+%5B%22driven%22%5D+%5B%22through%22%5D+%5B%22the%22%5D+%5B%22main%22%5D+%5B%22street%22%5D+%5B%5D+%5B%22the%22%5D+%5B%22Praggerstrasser%22%5D+%5B%22in%22%5D+%5B%22Dresden%22%5D+%5B%5D+%5B%22on%22%5D+%5B%22their%22%5D+%5B%22way%22%5D+%5B%22to%22%5D+%5B%22the%22%5D+%5B%22station%22%5D+%5B%22to%22%5D+%5B%22be%22%5D+%5B%22taken%22%5D+%5B%22to%22%5D+%5B%22the%22%5D+%5B%22concentration%22%5D+%5B%22camps%22%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': 'Jews were driven through the main street , the Praggerstrasser in Dresden , on their way to the station to be taken to the concentration camps ', 'right': '', 'complete_match': 'Jews were driven through the main street , the Praggerstrasser in Dresden , on their way to the station to be taken to the concentration camps ', 'testimony_id': 'irn506688', 'shelfmark': ['USHMM RG-50.030*0455'], 'token_start'

In [41]:
fragment_2 = {}
fragment_2['original_sentence'] = "A herd of humans are driven through deserted streets."
fragment_2['label']="A herd of humans are driven through deserted streets."
indices = find_sentence_id(fragment_2['original_sentence'])
fragment_2['start_sentence_index']=indices[0]
fragment_2['end_sentence_index']=indices[1]
fragment_2['media_offset'] = 0
fragment_2['media_index'] = 0
fragment_2['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_2)


http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22A%22%5D+%5B%22herd%22%5D+%5B%22of%22%5D+%5B%22humans%22%5D+%5B%22are%22%5D+%5B%22driven%22%5D+%5B%22through%22%5D+%5B%22deserted%22%5D+%5B%22streets%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': 'A herd of humans are driven through deserted streets . ', 'right': '', 'complete_match': 'A herd of humans are driven through deserted streets . ', 'testimony_id': 'usc_shoah_4345', 'shelfmark': ['USC Shoah Foundation 4345'], 'token_start': 26936, 'token_end': 26946}


In [42]:
fragment_3 = {}
fragment_3['original_sentence'] = "They put us in the center of the street where the cars and horses were driven, and they started to force us to march to the ghetto."
fragment_3['label']="They put us in the center of the street (..) and they started to force us to march to the ghetto."
indices = find_sentence_id(fragment_3['original_sentence'])
fragment_3['start_sentence_index']=indices[0]
fragment_3['end_sentence_index']=indices[1]
fragment_3['media_offset'] = 0
fragment_3['media_index'] = 0
fragment_3['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_3)

http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22They%22%5D+%5B%22put%22%5D+%5B%22us%22%5D+%5B%22in%22%5D+%5B%22the%22%5D+%5B%22center%22%5D+%5B%22of%22%5D+%5B%22the%22%5D+%5B%22street%22%5D+%5B%22where%22%5D+%5B%22the%22%5D+%5B%22cars%22%5D+%5B%22and%22%5D+%5B%22horses%22%5D+%5B%22were%22%5D+%5B%22driven%22%5D+%5B%5D+%5B%22and%22%5D+%5B%22they%22%5D+%5B%22started%22%5D+%5B%22to%22%5D+%5B%22force%22%5D+%5B%22us%22%5D+%5B%22to%22%5D+%5B%22march%22%5D+%5B%22to%22%5D+%5B%22the%22%5D+%5B%22ghetto%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': 'They put us in the center of the street where the cars and horses were driven , and they started to force us to march to the ghetto . ', 'right': '', 'complete_match': 'They put us in the center of the street where the cars and horses were driven , and they started to force us to march to the ghetto . ', 'testimony_id': 'usc_shoah_4361', 'shelfmark': ['USC Shoah Foundation 43

In [43]:
fragment_4 = {}
fragment_4['original_sentence'] = "people who were driven down the street, through-- to the-- to the edge of the forest and brutally murdered"
fragment_4['label']= "The people who were driven down the street, through to the edge of the forest and brutally murdered."
indices = find_sentence_id(fragment_4['original_sentence'])
fragment_4['start_sentence_index']=indices[0]
fragment_4['end_sentence_index']=indices[1]
fragment_4['media_offset'] = 0
fragment_4['media_index'] = 0
fragment_4['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_4)


http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22people%22%5D+%5B%22who%22%5D+%5B%22were%22%5D+%5B%22driven%22%5D+%5B%22down%22%5D+%5B%22the%22%5D+%5B%22street%22%5D+%5B%5D+%5B%22through%22%5D+%5B%5D%7B0%2C3%7D+%5B%22to%22%5D+%5B%22the%22%5D+%5B%5D%7B0%2C3%7D+%5B%22to%22%5D+%5B%22the%22%5D+%5B%22edge%22%5D+%5B%22of%22%5D+%5B%22the%22%5D+%5B%22forest%22%5D+%5B%22and%22%5D+%5B%22brutally%22%5D+%5B%22murdered%22%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': 'people who were driven down the street , through -- to the -- to the edge of the forest and brutally murdered ', 'right': '', 'complete_match': 'people who were driven down the street , through -- to the -- to the edge of the forest and brutally murdered ', 'testimony_id': 'usc_shoah_8916', 'shelfmark': ['USC Shoah Foundation 8916'], 'token_start': 6531, 'token_end': 6553}


In [44]:
add_testimonial_fragments(fragments)

### 4.  

In [45]:
lemmas = ["drive","cattle"]

In [46]:
query = create_contextual_query(lemmas,context_length=5)
print (query)

([lemma="drive"][]{0,5}[lemma="cattle"])|([lemma="cattle"][]{0,5}[lemma="drive"])


In [47]:
domain_term = "like cattle"

In [48]:
fragments = {}
fragments['main_node'] = main_node
fragments['mid_node'] = domain_term
fragments['fragments'] = []

In [49]:
fragment_1 = {}
fragment_1['original_sentence'] = "Like cattle we were driven into the gutter."
fragment_1['label']="Like cattle we were driven into the gutter."
indices = find_sentence_id(fragment_1['original_sentence'])
fragment_1['start_sentence_index']=indices[0]
fragment_1['end_sentence_index']=indices[1]
fragment_1['media_offset'] = 0
fragment_1['media_index'] = 0
fragment_1['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_1)

http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22Like%22%5D+%5B%22cattle%22%5D+%5B%22we%22%5D+%5B%22were%22%5D+%5B%22driven%22%5D+%5B%22into%22%5D+%5B%22the%22%5D+%5B%22gutter%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': 'Like cattle we were driven into the gutter . ', 'right': '', 'complete_match': 'Like cattle we were driven into the gutter . ', 'testimony_id': 'irn504712', 'shelfmark': ['USHMM RG-50.030*0220'], 'token_start': 1451, 'token_end': 1460}


In [50]:
fragment_2 = {}
fragment_2['original_sentence'] = "the aim was to drive us like cattle into Czernowitz, into the ghetto, in Czernowitz ghetto."
fragment_2['label']="(..)the aim was to drive us like cattle into Czernowitz, into the ghetto, in Czernowitz ghetto."
indices = find_sentence_id(fragment_2['original_sentence'])
fragment_2['start_sentence_index']=indices[0]
fragment_2['end_sentence_index']=indices[1]
fragment_2['media_offset'] = 0
fragment_2['media_index'] = 0
fragment_2['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_2)


http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22the%22%5D+%5B%22aim%22%5D+%5B%22was%22%5D+%5B%22to%22%5D+%5B%22drive%22%5D+%5B%22us%22%5D+%5B%22like%22%5D+%5B%22cattle%22%5D+%5B%22into%22%5D+%5B%22Czernowitz%22%5D+%5B%5D+%5B%22into%22%5D+%5B%22the%22%5D+%5B%22ghetto%22%5D+%5B%5D+%5B%22in%22%5D+%5B%22Czernowitz%22%5D+%5B%22ghetto%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': 'the aim was to drive us like cattle into Czernowitz , into the ghetto , in Czernowitz ghetto . ', 'right': '', 'complete_match': 'the aim was to drive us like cattle into Czernowitz , into the ghetto , in Czernowitz ghetto . ', 'testimony_id': 'irn508694', 'shelfmark': ['USHMM RG-50.462*0070'], 'token_start': 16280, 'token_end': 16299}


In [51]:
fragment_3 = {}
fragment_3['original_sentence'] = "They should be all driven away like cattle, you know."
fragment_3['label']="They should be all driven away like cattle, you know."
indices = find_sentence_id(fragment_3['original_sentence'])
fragment_3['start_sentence_index']=indices[0]
fragment_3['end_sentence_index']=indices[1]
fragment_3['media_offset'] = 0
fragment_3['media_index'] = 0
fragment_3['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_3)

http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22They%22%5D+%5B%22should%22%5D+%5B%22be%22%5D+%5B%22all%22%5D+%5B%22driven%22%5D+%5B%22away%22%5D+%5B%22like%22%5D+%5B%22cattle%22%5D+%5B%5D+%5B%22you%22%5D+%5B%22know%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': 'They should be all driven away like cattle , you know . ', 'right': '', 'complete_match': 'They should be all driven away like cattle , you know . ', 'testimony_id': 'usc_shoah_1641', 'shelfmark': ['USC Shoah Foundation 1641'], 'token_start': 17266, 'token_end': 17278}


In [52]:
fragment_4 = {}
fragment_4['original_sentence'] = "driven like cattle, with the valises, with the baggages, with my little nieces, with the little infant who was five, six weeks old."
fragment_4['label']= "(..)driven like cattle, with the valises, with the baggages, with my little nieces, with the little infant who was five, six weeks old."
indices = find_sentence_id(fragment_4['original_sentence'])
fragment_4['start_sentence_index']=indices[0]
fragment_4['end_sentence_index']=indices[1]
fragment_4['media_offset'] = 0
fragment_4['media_index'] = 0
fragment_4['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_4)


http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22driven%22%5D+%5B%22like%22%5D+%5B%22cattle%22%5D+%5B%5D+%5B%22with%22%5D+%5B%22the%22%5D+%5B%22valises%22%5D+%5B%5D+%5B%22with%22%5D+%5B%22the%22%5D+%5B%22baggages%22%5D+%5B%5D+%5B%22with%22%5D+%5B%22my%22%5D+%5B%22little%22%5D+%5B%22nieces%22%5D+%5B%5D+%5B%22with%22%5D+%5B%22the%22%5D+%5B%22little%22%5D+%5B%22infant%22%5D+%5B%22who%22%5D+%5B%22was%22%5D+%5B%22five%22%5D+%5B%5D+%5B%22six%22%5D+%5B%22weeks%22%5D+%5B%22old%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': 'driven like cattle , with the valises , with the baggages , with my little nieces , with the little infant who was five , six weeks old . ', 'right': '', 'complete_match': 'driven like cattle , with the valises , with the baggages , with my little nieces , with the little infant who was five , six weeks old . ', 'testimony_id': 'usc_shoah_4361', 'shelfmark': ['USC Shoah Foundation 4361'], 'token_sta

In [53]:
fragment_5 = {}
fragment_5['original_sentence'] = "they're driven, literally driven like cattle, from the Riga ghetto "
fragment_5['label']= "(..)they're driven, literally driven like cattle, from the Riga ghetto (..)."
indices = find_sentence_id(fragment_5['original_sentence'])
fragment_5['start_sentence_index']=indices[0]
fragment_5['end_sentence_index']=indices[1]
fragment_5['media_offset'] = 0
fragment_5['media_index'] = 0
fragment_5['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_5)

http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22they%22%5D+%5B%5D+%5B%22driven%22%5D+%5B%5D+%5B%22literally%22%5D+%5B%22driven%22%5D+%5B%22like%22%5D+%5B%22cattle%22%5D+%5B%5D+%5B%22from%22%5D+%5B%22the%22%5D+%5B%22Riga%22%5D+%5B%22ghetto%22%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': "they 're driven , literally driven like cattle , from the Riga ghetto ", 'right': '', 'complete_match': "they 're driven , literally driven like cattle , from the Riga ghetto ", 'testimony_id': 'usc_shoah_9538', 'shelfmark': ['USC Shoah Foundation 9538'], 'token_start': 7263, 'token_end': 7276}


In [54]:
add_testimonial_fragments(fragments)

### 5.  

In [55]:
lemmas = ["drive","barracks"]

In [56]:
query = create_contextual_query(lemmas,context_length=50)
print (query)

([lemma="drive"][]{0,50}[lemma="barracks"])|([lemma="barracks"][]{0,50}[lemma="drive"])


In [57]:
domain_term = "barracks"

In [58]:
fragments = {}
fragments['main_node'] = main_node
fragments['mid_node'] = domain_term
fragments['fragments'] = []

In [59]:
fragment_1 = {}
fragment_1['original_sentence'] = "And then we were marched into the camp itself. We were driven into barracks."
fragment_1['label']="And then we were marched into the camp itself. We were driven into barracks."
indices = find_sentence_id(fragment_1['original_sentence'])
fragment_1['start_sentence_index']=indices[0]
fragment_1['end_sentence_index']=indices[1]
fragment_1['media_offset'] = 0
fragment_1['media_index'] = 0
fragment_1['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_1)

http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22And%22%5D+%5B%22then%22%5D+%5B%22we%22%5D+%5B%22were%22%5D+%5B%22marched%22%5D+%5B%22into%22%5D+%5B%22the%22%5D+%5B%22camp%22%5D+%5B%22itself%22%5D+%5B%5D+%5B%22We%22%5D+%5B%22were%22%5D+%5B%22driven%22%5D+%5B%22into%22%5D+%5B%22barracks%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': 'And then we were marched into the camp itself . We were driven into barracks . ', 'right': '', 'complete_match': 'And then we were marched into the camp itself . We were driven into barracks . ', 'testimony_id': 'usc_shoah_2916', 'shelfmark': ['USC Shoah Foundation 2916'], 'token_start': 13107, 'token_end': 13123}


In [60]:
fragment_2 = {}
fragment_2['original_sentence'] = "We were driven back into the barracks he was behind me. Because when we were driven in, we were driven in, literally by-- by with beatings, gun butts."
fragment_2['label']="We were driven back into the barracks (..) we were driven in, literally by-- by with beatings, gun butts."
indices = find_sentence_id(fragment_2['original_sentence'])
fragment_2['start_sentence_index']=indices[0]
fragment_2['end_sentence_index']=indices[1]
fragment_2['media_offset'] = 0
fragment_2['media_index'] = 0
fragment_2['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_2)


http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22We%22%5D+%5B%22were%22%5D+%5B%22driven%22%5D+%5B%22back%22%5D+%5B%22into%22%5D+%5B%22the%22%5D+%5B%22barracks%22%5D+%5B%22he%22%5D+%5B%22was%22%5D+%5B%22behind%22%5D+%5B%22me%22%5D+%5B%5D+%5B%22Because%22%5D+%5B%22when%22%5D+%5B%22we%22%5D+%5B%22were%22%5D+%5B%22driven%22%5D+%5B%22in%22%5D+%5B%5D+%5B%22we%22%5D+%5B%22were%22%5D+%5B%22driven%22%5D+%5B%22in%22%5D+%5B%5D+%5B%22literally%22%5D+%5B%22by%22%5D+%5B%5D%7B0%2C3%7D+%5B%22by%22%5D+%5B%22with%22%5D+%5B%22beatings%22%5D+%5B%5D+%5B%22gun%22%5D+%5B%22butts%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': 'we were driven back into the barracks he was behind me . Because when we were driven in , we were driven in , literally by -- by with beatings , gun butts . ', 'right': '', 'complete_match': 'we were driven back into the barracks he was behind me . Because when we were driven in , we were driven in , literally b

In [61]:
fragment_3 = {}
fragment_3['original_sentence'] = "this was a demoralizing process, and I think that we were demoralized by the second, I mean you moved with it, it was as though we were driven in into barracks."
fragment_3['label']="(..) this was a demoralizing process (..) it was as though we were driven in into barracks."
indices = find_sentence_id(fragment_3['original_sentence'])
fragment_3['start_sentence_index']=indices[0]
fragment_3['end_sentence_index']=indices[1]
fragment_3['media_offset'] = 0
fragment_3['media_index'] = 0
fragment_3['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_3)

http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22this%22%5D+%5B%22was%22%5D+%5B%22a%22%5D+%5B%22demoralizing%22%5D+%5B%22process%22%5D+%5B%5D+%5B%22and%22%5D+%5B%22I%22%5D+%5B%22think%22%5D+%5B%22that%22%5D+%5B%22we%22%5D+%5B%22were%22%5D+%5B%22demoralized%22%5D+%5B%22by%22%5D+%5B%22the%22%5D+%5B%22second%22%5D+%5B%5D+%5B%22I%22%5D+%5B%22mean%22%5D+%5B%22you%22%5D+%5B%22moved%22%5D+%5B%22with%22%5D+%5B%22it%22%5D+%5B%5D+%5B%22it%22%5D+%5B%22was%22%5D+%5B%22as%22%5D+%5B%22though%22%5D+%5B%22we%22%5D+%5B%22were%22%5D+%5B%22driven%22%5D+%5B%22in%22%5D+%5B%22into%22%5D+%5B%22barracks%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': 'this was a demoralizing process , and I think that we were demoralized by the second , I mean you moved with it , it was as though we were driven in into barracks . ', 'right': '', 'complete_match': 'this was a demoralizing process , and I think that we were demoralized by the second , I 

In [62]:
fragment_4 = {}
fragment_4['original_sentence'] = "They drove us naked from the barracks for 300 feet."
fragment_4['label']= "They drove us naked from the barracks for 300 feet."
indices = find_sentence_id(fragment_4['original_sentence'])
fragment_4['start_sentence_index']=indices[0]
fragment_4['end_sentence_index']=indices[1]
fragment_4['media_offset'] = 0
fragment_4['media_index'] = 0
fragment_4['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_4)


http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22They%22%5D+%5B%22drove%22%5D+%5B%22us%22%5D+%5B%22naked%22%5D+%5B%22from%22%5D+%5B%22the%22%5D+%5B%22barracks%22%5D+%5B%22for%22%5D+%5B%22300%22%5D+%5B%22feet%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': 'They drove us naked from the barracks for 300 feet . ', 'right': '', 'complete_match': 'They drove us naked from the barracks for 300 feet . ', 'testimony_id': 'irn507492', 'shelfmark': ['USHMM RG-50.031*0062'], 'token_start': 3212, 'token_end': 3223}


In [63]:
fragment_5 = {}
fragment_5['original_sentence'] = "we were driven into a barrack. And we came into the barrack, was chloride up to the knees."
fragment_5['label']= "(..) we were driven into a barrack. And we came into the barrack, was chloride up to the knees."
indices = find_sentence_id(fragment_5['original_sentence'])
fragment_5['start_sentence_index']=indices[0]
fragment_5['end_sentence_index']=indices[1]
fragment_5['media_offset'] = 0
fragment_5['media_index'] = 0
fragment_5['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_5)

http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22we%22%5D+%5B%22were%22%5D+%5B%22driven%22%5D+%5B%22into%22%5D+%5B%22a%22%5D+%5B%22barrack%22%5D+%5B%5D+%5B%22And%22%5D+%5B%22we%22%5D+%5B%22came%22%5D+%5B%22into%22%5D+%5B%22the%22%5D+%5B%22barrack%22%5D+%5B%5D+%5B%22was%22%5D+%5B%22chloride%22%5D+%5B%22up%22%5D+%5B%22to%22%5D+%5B%22the%22%5D+%5B%22knees%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': 'we were driven into a barrack . And we came into the barrack , was chloride up to the knees . ', 'right': '', 'complete_match': 'we were driven into a barrack . And we came into the barrack , was chloride up to the knees . ', 'testimony_id': 'usc_shoah_27390', 'shelfmark': ['USC Shoah Foundation 27390'], 'token_start': 26230, 'token_end': 26251}


In [64]:
add_testimonial_fragments(fragments)

### 6.  

In [65]:
lemmas = ["drive","river"]

In [66]:
query = create_contextual_query(lemmas,context_length=10)
print (query)

([lemma="drive"][]{0,10}[lemma="river"])|([lemma="river"][]{0,10}[lemma="drive"])


In [67]:
domain_term = "river"

In [68]:
fragments = {}
fragments['main_node'] = main_node
fragments['mid_node'] = domain_term
fragments['fragments'] = []

In [69]:
fragment_1 = {}
fragment_1['original_sentence'] = "They drove them down into a river bank where they could control that you couldn't run away."
fragment_1['label']="They drove them down into a river bank where they could control that you couldn't run away."
indices = find_sentence_id(fragment_1['original_sentence'])
fragment_1['start_sentence_index']=indices[0]
fragment_1['end_sentence_index']=indices[1]
fragment_1['media_offset'] = 0
fragment_1['media_index'] = 0
fragment_1['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_1)

http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22They%22%5D+%5B%22drove%22%5D+%5B%22them%22%5D+%5B%22down%22%5D+%5B%22into%22%5D+%5B%22a%22%5D+%5B%22river%22%5D+%5B%22bank%22%5D+%5B%22where%22%5D+%5B%22they%22%5D+%5B%22could%22%5D+%5B%22control%22%5D+%5B%22that%22%5D+%5B%22you%22%5D+%5B%22could%22%5D+%5B%5D+%5B%22run%22%5D+%5B%22away%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': "They drove them down into a river bank where they could control that you could n't run away . ", 'right': '', 'complete_match': "They drove them down into a river bank where they could control that you could n't run away . ", 'testimony_id': 'irn504718', 'shelfmark': ['USHMM RG-50.030*0228'], 'token_start': 4351, 'token_end': 4370}


In [70]:
fragment_2 = {}
fragment_2['original_sentence'] = "And next day, they were driven-- they were chased over the river and shot. All the people."
fragment_2['label']="(..) they were driven-- they were chased over the river and shot. All the people."
indices = find_sentence_id(fragment_2['original_sentence'])
fragment_2['start_sentence_index']=indices[0]
fragment_2['end_sentence_index']=indices[1]
fragment_2['media_offset'] = 0
fragment_2['media_index'] = 0
fragment_2['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_2)


http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22And%22%5D+%5B%22next%22%5D+%5B%22day%22%5D+%5B%5D+%5B%22they%22%5D+%5B%22were%22%5D+%5B%22driven%22%5D+%5B%5D%7B0%2C3%7D+%5B%22they%22%5D+%5B%22were%22%5D+%5B%22chased%22%5D+%5B%22over%22%5D+%5B%22the%22%5D+%5B%22river%22%5D+%5B%22and%22%5D+%5B%22shot%22%5D+%5B%5D+%5B%22All%22%5D+%5B%22the%22%5D+%5B%22people%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': 'And next day , they were driven -- they were chased over the river and shot . All the people . ', 'right': '', 'complete_match': 'And next day , they were driven -- they were chased over the river and shot . All the people . ', 'testimony_id': 'HVT-181', 'shelfmark': ['Fortunoff Archive HVT-181'], 'token_start': 8009, 'token_end': 8030}


In [71]:
fragment_3 = {}
fragment_3['original_sentence'] = "They were driven into the river and shot in the back with machine guns."
fragment_3['label']="They were driven into the river and shot in the back with machine guns."
indices = find_sentence_id(fragment_3['original_sentence'])
fragment_3['start_sentence_index']=indices[0]
fragment_3['end_sentence_index']=indices[1]
fragment_3['media_offset'] = 0
fragment_3['media_index'] = 0
fragment_3['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_3)

http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22They%22%5D+%5B%22were%22%5D+%5B%22driven%22%5D+%5B%22into%22%5D+%5B%22the%22%5D+%5B%22river%22%5D+%5B%22and%22%5D+%5B%22shot%22%5D+%5B%22in%22%5D+%5B%22the%22%5D+%5B%22back%22%5D+%5B%22with%22%5D+%5B%22machine%22%5D+%5B%22guns%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': 'They were driven into the river and shot in the back with machine guns . ', 'right': '', 'complete_match': 'They were driven into the river and shot in the back with machine guns . ', 'testimony_id': 'usc_shoah_945', 'shelfmark': ['USC Shoah Foundation 945'], 'token_start': 10051, 'token_end': 10066}


In [72]:
add_testimonial_fragments(fragments)

### 7.  

In [73]:
lemmas = ["drive","beat"]

In [74]:
query = create_contextual_query(lemmas,context_length=10)
print (query)

([lemma="drive"][]{0,10}[lemma="beat"])|([lemma="beat"][]{0,10}[lemma="drive"])


In [75]:
domain_term = "beat"

In [76]:
fragments = {}
fragments['main_node'] = main_node
fragments['mid_node'] = domain_term
fragments['fragments'] = []

In [77]:
fragment_1 = {}
fragment_1['original_sentence'] = "They were driving us like you would drive a sheep, animals. Beating on us on the way."
fragment_1['label']="They were driving us like you would drive a sheep, animals. Beating on us on the way."
indices = find_sentence_id(fragment_1['original_sentence'])
fragment_1['start_sentence_index']=indices[0]
fragment_1['end_sentence_index']=indices[1]
fragment_1['media_offset'] = 0
fragment_1['media_index'] = 0
fragment_1['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_1)

http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22They%22%5D+%5B%22were%22%5D+%5B%22driving%22%5D+%5B%22us%22%5D+%5B%22like%22%5D+%5B%22you%22%5D+%5B%22would%22%5D+%5B%22drive%22%5D+%5B%22a%22%5D+%5B%22sheep%22%5D+%5B%5D+%5B%22animals%22%5D+%5B%5D+%5B%22Beating%22%5D+%5B%22on%22%5D+%5B%22us%22%5D+%5B%22on%22%5D+%5B%22the%22%5D+%5B%22way%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': 'They were driving us like you would drive a sheep , animals . Beating on us on the way . ', 'right': '', 'complete_match': 'They were driving us like you would drive a sheep , animals . Beating on us on the way . ', 'testimony_id': 'irn504860', 'shelfmark': ['USHMM RG-50.030*0367'], 'token_start': 18268, 'token_end': 18288}


In [78]:
fragment_2 = {}
fragment_2['original_sentence'] = "with the whip, they were just beating them and-- and driving them."
fragment_2['label']="(..) with the whip, they were just beating them and and driving them."
indices = find_sentence_id(fragment_2['original_sentence'])
fragment_2['start_sentence_index']=indices[0]
fragment_2['end_sentence_index']=indices[1]
fragment_2['media_offset'] = 0
fragment_2['media_index'] = 0
fragment_2['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_2)


http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22with%22%5D+%5B%22the%22%5D+%5B%22whip%22%5D+%5B%5D+%5B%22they%22%5D+%5B%22were%22%5D+%5B%22just%22%5D+%5B%22beating%22%5D+%5B%22them%22%5D+%5B%22and%22%5D+%5B%5D%7B0%2C3%7D+%5B%22and%22%5D+%5B%22driving%22%5D+%5B%22them%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': 'with the whip , they were just beating them and -- and driving them . ', 'right': '', 'complete_match': 'with the whip , they were just beating them and -- and driving them . ', 'testimony_id': 'usc_shoah_182', 'shelfmark': ['USC Shoah Foundation 182'], 'token_start': 5952, 'token_end': 5967}


In [79]:
fragment_3 = {}
fragment_3['original_sentence'] = "they were being driven and beaten into the, into the cars"
fragment_3['label']="(..) they were being driven and beaten into the, into the cars (..)"
indices = find_sentence_id(fragment_3['original_sentence'])
fragment_3['start_sentence_index']=indices[0]
fragment_3['end_sentence_index']=indices[1]
fragment_3['media_offset'] = 0
fragment_3['media_index'] = 0
fragment_3['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_3)

http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22they%22%5D+%5B%22were%22%5D+%5B%22being%22%5D+%5B%22driven%22%5D+%5B%22and%22%5D+%5B%22beaten%22%5D+%5B%22into%22%5D+%5B%22the%22%5D+%5B%5D+%5B%22into%22%5D+%5B%22the%22%5D+%5B%22cars%22%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': 'they were being driven and beaten into the , into the cars ', 'right': '', 'complete_match': 'they were being driven and beaten into the , into the cars ', 'testimony_id': 'usc_shoah_19550', 'shelfmark': ['USC Shoah Foundation 19550'], 'token_start': 19335, 'token_end': 19347}


In [80]:
fragment_4 = {}
fragment_4['original_sentence'] = "drive people through the streets, beat them,"
fragment_4['label']= "(..)drive people through the streets, beat them(..)"
indices = find_sentence_id(fragment_4['original_sentence'])
fragment_4['start_sentence_index']=indices[0]
fragment_4['end_sentence_index']=indices[1]
fragment_4['media_offset'] = 0
fragment_4['media_index'] = 0
fragment_4['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_4)


http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22drive%22%5D+%5B%22people%22%5D+%5B%22through%22%5D+%5B%22the%22%5D+%5B%22streets%22%5D+%5B%5D+%5B%22beat%22%5D+%5B%22them%22%5D+%5B%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': 'drive people through the streets , beat them , ', 'right': '', 'complete_match': 'drive people through the streets , beat them , ', 'testimony_id': 'irn505558', 'shelfmark': ['USHMM RG-50.042*0004'], 'token_start': 384, 'token_end': 393}


In [81]:
fragment_5 = {}
fragment_5['original_sentence'] = "they were beating the people and beating them with sticks and with whips and driving with screams and shouts"
fragment_5['label']= "(..)beating them with sticks and with whips and driving with screams and shouts(..)."
indices = find_sentence_id(fragment_5['original_sentence'])
fragment_5['start_sentence_index']=indices[0]
fragment_5['end_sentence_index']=indices[1]
fragment_5['media_offset'] = 0
fragment_5['media_index'] = 0
fragment_5['testimony_id'] = indices[2]
fragments['fragments'].append(fragment_5)

http://localhost:8080/blacklab-server-2.1.0/lts/hits?patt=%5B%22they%22%5D+%5B%22were%22%5D+%5B%22beating%22%5D+%5B%22the%22%5D+%5B%22people%22%5D+%5B%22and%22%5D+%5B%22beating%22%5D+%5B%22them%22%5D+%5B%22with%22%5D+%5B%22sticks%22%5D+%5B%22and%22%5D+%5B%22with%22%5D+%5B%22whips%22%5D+%5B%22and%22%5D+%5B%22driving%22%5D+%5B%22with%22%5D+%5B%22screams%22%5D+%5B%22and%22%5D+%5B%22shouts%22%5D&waitfortotal=true&outputformat=json&prettyprint=no&wordsaroundhit=0
{'left': '', 'match_word': 'they were beating the people and beating them with sticks and with whips and driving with screams and shouts ', 'right': '', 'complete_match': 'they were beating the people and beating them with sticks and with whips and driving with screams and shouts ', 'testimony_id': 'usc_shoah_27507', 'shelfmark': ['USC Shoah Foundation 27507'], 'token_start': 26213, 'token_end': 26232}


In [82]:
add_testimonial_fragments(fragments)